# Example: Compute the Primal and Dual Solution for a Reaction Flow Problem
This example will familiarize students with using [linear programming](https://en.wikipedia.org/wiki/Linear_programming) to compute the `reaction flow` or `flux` through a chemical reaction network. 

## Setup
This example requires several external libraries and a function to compute the outer product. Let's download and install these packages and call our `Include.jl` file.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-12/L12a`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-12/L12a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-12/L12a/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-12/L12a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-12/L12a/Manifest.toml`


## Prerequisites 
Before we can compute the primal and dual solutions to the flux problem, we need to load the stoichiometric matrix from the reaction file, in this case [Toy.net](data/Toy.net), i.e., the same example reaction network that we used for `PS3`.

In [2]:
# Load the reaction file -
path_to_reaction_file = joinpath(_PATH_TO_DATA, "Toy.net");
R = readreactionfile(path_to_reaction_file, comment="//", delim=',', expand = false);

# build the stoichiometric matrix -
S = build(MyStoichiometricMatrixModel, R);
number_of_species = length(S.species)

measured_columns = [2]; # we have measured these columns (product formation)
unkown_columns = setdiff(1:24, measured_columns); # we want to estimate these columns

# build the system matrix -
d = [1,1,-1]
A = [S.matrix d[1]*diagm(ones(number_of_species)) d[2]*diagm(ones(number_of_species)) d[3]*diagm(ones(number_of_species))];

Â = A[:,unkown_columns];
B = A[:,measured_columns];

In [3]:
measurement = [2.0]; # we want a unit of product formation

In [4]:
stream_label_vector = [
    "ϵ̇₁", "ϵ̇₂", "ϵ̇₃", 
    "ṅ_A1_1", "ṅ_A2_1", "ṅ_B_1", "ṅ_C_1", "ṅ_P_1", "ṅ_x_1", "ṅ_y_1",
    "ṅ_A1_2", "ṅ_A2_2", "ṅ_B_2", "ṅ_C_2", "ṅ_P_2", "ṅ_x_2", "ṅ_y_2",
    "ṅ_A1_3", "ṅ_A2_3", "ṅ_B_3", "ṅ_C_3", "ṅ_P_3", "ṅ_x_3", "ṅ_y_3"
];
label_vector = stream_label_vector[unkown_columns]

23-element Vector{String}:
 "ϵ̇₁"
 "ϵ̇₃"
 "ṅ_A1_1"
 "ṅ_A2_1"
 "ṅ_B_1"
 "ṅ_C_1"
 "ṅ_P_1"
 "ṅ_x_1"
 "ṅ_y_1"
 "ṅ_A1_2"
 "ṅ_A2_2"
 "ṅ_B_2"
 "ṅ_C_2"
 "ṅ_P_2"
 "ṅ_x_2"
 "ṅ_y_2"
 "ṅ_A1_3"
 "ṅ_A2_3"
 "ṅ_B_3"
 "ṅ_C_3"
 "ṅ_P_3"
 "ṅ_x_3"
 "ṅ_y_3"

## `Primal`: Compute the primal flux solution
Fill me in

In [5]:
(number_of_species, number_of_flows) = size(Â);

In [6]:
c = -1*ones(number_of_flows); # we want to *minimize* the material usage 

### Setup the bounds on the flows

In [7]:
bounds_primal = [

    # --- reactions ---
    0.0 100.0 ; # 1 1 ϵ̇₁
    # 0.0 100.0 ; # 2 ϵ̇₂ we measured this
    0.0 100.0 ; # 3 1 ϵ̇₃
    
    # --- stream 1 -----
    0.0 10.0 ; # 4 s1 A1
    0.0 6.0  ; # 5 s1 A2
    0.0 0.0  ; # 6 s1 B
    0.0 0.0  ; # 7 s1 C
    0.0 0.0  ; # 8 s1 P
    0.0 0.0  ; # 9 s1 x
    0.0 0.0  ; # 10 s1 y

    # --- stream 2 -----
    0.0 0.0  ; # 11 s2 A1
    0.0 0.0  ; # 12 s2 A2
    0.0 0.0  ; # 13 s2 B
    0.0 0.0  ; # 14 s2 C
    0.0 0.0  ; # 15 s2 P
    0.0 10.0  ; # 16 s2 x
    0.0 0.0  ; # 17 s2 y

    # --- stream 3 (unbounded)
    0.0 1000.0  ; # 18 s2 A1
    0.0 1000.0  ; # 19 s2 A2
    0.0 1000.0  ; # 20 s2 B
    0.0 1000.0  ; # 21 s2 C
    0.0 1000.0  ; # 22 s2 P
    0.0 1000.0  ; # 23 s2 x
    0.0 1000.0  ; # 24 s2 y
];

In [8]:
primal_problem = build(MyLinearProgrammingProblemModel, (
    c = c,
    A = Â,
    b = -B*measurement,
    lb = bounds_primal[:,1],
    ub = bounds_primal[:,2]
));

In [9]:
primal_soln = solve(primal_problem)

Dict{String, Any} with 2 entries:
  "argmax"          => [2.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.…
  "objective_value" => -10.0

In [10]:
primal_soln_table = DataFrame();
primal_flow = primal_soln["argmax"]; # get the primal solution from the solution dictionary
for i ∈ eachindex(unkown_columns)
    row_data = (
        label=label_vector[i],
        value = primal_flow[i]
    );
    push!(primal_soln_table, row_data);
end
primal_soln_table

Row,label,value
,String,Float64
1,ϵ̇₁,2.0
2,ϵ̇₃,0.0
3,ṅ_A1_1,2.0
4,ṅ_A2_1,0.0
5,ṅ_B_1,0.0
6,ṅ_C_1,0.0
7,ṅ_P_1,0.0
8,ṅ_x_1,0.0
9,ṅ_y_1,0.0


## `Dual`: Compute the dual flux solution

In [11]:
dual_problem = build(MyLinearProgrammingDualProblemModel, (
    c = c,
    A = Â,
    b = -B*measurement,
    lb = zeros(number_of_flows),
    ub = 1000*ones(number_of_flows)
));

In [12]:
dual_soln = solve(dual_problem)

Dict{String, Any} with 2 entries:
  "argmin"          => [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
  "objective_value" => -2.0

In [13]:
dual_flow = dual_soln["argmin"];
[S.species dual_flow]

7×2 Matrix{Any}:
 "A1"  0.0
 "A2"  0.0
 "B"   0.0
 "C"   0.0
 "P"   1.0
 "x"   0.0
 "y"   0.0